In [33]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from spherecluster import SphericalKMeans
import statsmodels.api as sm

In [34]:
# data_full = pd.read_excel('3d_FinalResult_0723(trainonly2012).xlsx')
data_full = pd.read_excel('2d_FinalResult_0510.xlsx')

ff49_index= pd.read_csv('Siccodes49.csv')

In [40]:
def ff49_class(sic):
    target_data = ff49_index[(ff49_index['sic1'] <= sic) & (ff49_index['sic2'] >= sic)]
    if len(target_data) == 0:
        ff_code = 49
    else:
        ff_code = target_data['ff49'].values[0]
    return ff_code

def ff12_class(sic):
    industry_code = 10
    if ((100 <= sic <= 999) or (2000 <= sic <= 2399) or (2700 <= sic <= 2749) or (2770 <= sic <= 2799) or (3100 <= sic <= 3199) or (3940 <= sic <= 3989)):
        industry_code =  1 # Consumer Non Durable
    elif ((2500 <= sic <= 2519) or (2590 <= sic <= 2599) or (3630 <= sic <= 3659) or (3710 <= sic <= 3711) or 
          (3714 <= sic <= 3714) or (3716 <= sic <= 3716) or (3750 <= sic <= 3751) or (3792 <= sic <= 3792) or (3900 <= sic <= 3939) or (3990 <= sic <= 3999)):
        industry_code =  2 # Consumer Durable
    elif ((2520 <= sic <= 2589) or (2600 <= sic <= 2699) or (2750 <= sic <= 2769) or (3000 <= sic <= 3099) or 
          (3200 <= sic <= 3569) or (3580 <= sic <= 3621) or (3700 <= sic <= 3709) or (3712 <= sic <= 3713) or (3715 <= sic <= 3715) or 
          (3717 <= sic <= 3749) or (3752 <= sic <= 3791) or (3793 <= sic <= 3799) or (3830 <= sic <= 3839) or (3860 <= sic <= 3899)):
        industry_code = 3 # Manufacturing
    elif ((1200 <= sic <= 1399) or (2900 <= sic <= 2999)):
        industry_code = 4 # Energy
    elif ((2800 <= sic <= 2829) or (2840 <= sic <= 2899)):
        industry_code = 5 # Chemicals
    elif ((3570 <= sic <= 3579) or (3660 <= sic <= 3692) or (3694 <= sic <= 3699) or (3810 <= sic <= 3829) or (7370 <= sic <= 7379)):
        industry_code = 6 # Business equipment
    elif (4800 <= sic <= 4899):
        industry_code = 7 # Telecom
    elif (4900 <= sic <= 4949):
        industry_code = 8 # Utilities
    elif ((5000 <= sic <= 5999) or (7200 <= sic <= 7299) or (7600 <= sic <= 7699)):
        industry_code = 9 # Shops Wholesale (Retail)
    elif ((2830 <= sic <= 2839) or (3693 <= sic <= 3693) or (3840 <= sic <= 3859) or (8000 <= sic <= 8099)):
        industry_code = 10 # Healthcare, medical, drugs
    elif (6000 <= sic <= 6999):
        industry_code = 11 # Money Finance
    else:
        industry_code = 12 # Others
        
    return industry_code


data_full['ff49'] = data_full['sic'].apply(lambda sic: ff49_class(sic))
data_full['ff12'] = data_full['sic'].apply(lambda sic: ff12_class(sic))
data_full['naics_sector'] = data_full['naics'].astype('str').str[:2]


arange = list(range(0,2))
num_clusters = 12

# 이 부분을 바꿔야함 (2012년도 데이터만 가지고 fit 해서 전체 데이터로 predict하는 형식으로)
X_cart = data_full[arange]
kmeans_labels = SphericalKMeans(num_clusters,normalize=False,random_state=0).fit_predict(X_cart)
data_full['spe_kmeans'] = kmeans_labels
data_full['spe_kmeans'] = data_full['spe_kmeans'].astype('int')
print(data_full.shape)
data_full.head()

(18072, 19)


,0,1,company,sic,naics,fyear,gvkey,exchange,ex,cik,cusip,gind,gsector,ff49,ff12,naics_sector,spe_kmeans,sic_changed,changed
4131,-1.444362,120.454338,AAR CORP,5080.0,423860.0,2013,1004.0,NYSE,NYSE,1750,000361105,201010.0,20.0,42,9,42,4,0.0,0.0
8842,-2.137872,121.118759,AAR CORP,5080.0,423860.0,2014,1004.0,NYSE,NYSE,1750,000361105,201010.0,20.0,42,9,42,4,0.0,0.0
13414,1.786478,129.490906,AAR CORP,5080.0,423860.0,2015,1004.0,NYSE,NYSE,1750,000361105,201010.0,20.0,42,9,42,4,0.0,0.0
17654,4.976506,127.102615,AAR CORP,5080.0,423860.0,2016,1004.0,NYSE,NYSE,1750,000361105,201010.0,20.0,42,9,42,4,0.0,0.0
2153,751.863953,145.399261,AMERICAN AIRLINES GROUP INC,4512.0,481111.0,2012,1045.0,NASDAQ,NASDAQ,6201,02376R102,203020.0,20.0,41,12,48,6,0.0,0.0


In [41]:
data_full.sort_values(by=['gvkey','fyear'], inplace=True)
data_full['sic_changed'] = data_full.groupby(by='gvkey')['sic'].diff().fillna(0)

changed = data_full[data_full['sic_changed'] != 0]
changed_firm = changed['gvkey'].unique()
changed_firm.shape[0]

214

In [42]:
def table_for(cls):
    data_full.sort_values(by=['gvkey','fyear'], inplace=True)
    data_full['changed']=data_full.groupby('gvkey')[cls].diff().fillna(0)
    
    ff12_changed = data_full[data_full['changed'] != 0]
    changed_list = data_full[data_full['gvkey'].isin(ff12_changed['gvkey'].unique())]
    changed_list['prev'] = np.where(changed_list['changed'] ==0, changed_list[cls], changed_list[cls] - changed_list['changed'])
    
    essential = changed_list[changed_list[cls] != changed_list['prev']]
    ctab = pd.crosstab(essential['prev'], essential[cls], margins=True)
    return essential, ctab

In [43]:
tab, ctab = table_for('ff12')
ctab

c:\program files\anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ff12,1,2,3,4,5,6,7,9,10,11,12,All
prev,,,,,,,,,,,,
1.0,0,0,0,0,0,1,1,1,0,2,0,5
2.0,0,0,2,0,1,0,0,0,0,0,1,4
3.0,2,1,0,1,0,9,0,0,0,1,2,16
4.0,0,0,0,0,1,1,0,1,1,1,0,5
5.0,0,0,0,0,0,1,0,0,1,0,1,3
6.0,1,0,5,0,0,0,2,3,4,2,6,23
7.0,0,0,2,0,0,2,0,0,0,1,1,6
9.0,1,1,0,1,0,0,0,0,0,2,2,7
10.0,0,0,0,0,0,0,0,1,0,1,0,2


In [44]:
# tab[(tab['prev'] == 11) & (tab['ff12'] == 6)].to_excel('11to6.xlsx')

In [45]:
tab, ctab = table_for('spe_kmeans')
ctab

c:\program files\anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


spe_kmeans,0,1,2,3,4,5,6,7,8,9,10,11,All
prev,,,,,,,,,,,,,
0.0,0,1,2,8,18,11,5,1,0,10,3,2,61
1.0,2,0,4,2,2,9,4,21,0,0,44,4,92
2.0,0,1,0,6,2,9,11,2,4,0,49,19,103
3.0,12,2,4,0,36,38,7,2,2,6,3,3,115
4.0,17,7,3,38,0,16,5,6,1,0,6,3,102
5.0,7,5,11,37,19,0,33,14,3,7,16,5,157
6.0,3,3,7,5,12,23,0,41,1,2,1,3,101
7.0,4,19,4,6,3,23,33,0,1,2,5,0,100
8.0,1,0,8,1,4,0,1,0,0,17,1,18,51


In [26]:
# Load business parts
import pickle
with open('comp_sic_merged_0510.pickle', 'rb') as f:
    comp_sic_data = pickle.load(f)

business_part = comp_sic_data['business_part']
company = comp_sic_data['company']
sic = comp_sic_data['sic']
naics = comp_sic_data['naics']
# snms = comp_sic_data['snms']
fyear = comp_sic_data['fyear']
exchange = comp_sic_data['exchange']
gvkey = comp_sic_data['gvkey']
cik = comp_sic_data['cik']
ex = comp_sic_data['ex']
cusip = comp_sic_data['cusip']
gind = comp_sic_data['gind']
gsector = comp_sic_data['gsector']